# Retrevial Agumented Generation (RAG)

Keeping large amounts of embedded data in a database and retreving the relevant informaiton for answering a question

### Indexing : Document loading with PyPDF Loader

In [99]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import CharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

import numpy as np

import copy
import dotenv

dotenv.load_dotenv()


True

In [2]:
loader_pdf = PyPDFLoader("Documents_for_indexing/introduction_to_Data_and_Data_Science.pdf")

In [3]:
page_pdf = loader_pdf.load()

In [4]:
page_pdf

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'Documents_for_indexing/introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Analysis vs Analytics \nAlright! So… \nLet’s discuss the not-so-obvious differences \nbetween the terms analysis and analytics. \nDue to the similarity of the words, some people \nbelieve they share the same meaning, and thus \nuse them interchangeably. Technically, this \nisn’t correct. There is, in fact, a distinct \ndifference between the two. And the reason \nfor one often being used instead of the other \nis the lack of a transparent understanding \nof both. \nSo, let’s clear this up, shall we? \nFirst, we will start with analysis. \nConsider the following… \nYou have a huge dataset containing data of \nvarious t

In [5]:
len(page_pdf)

6

In [6]:
pages_pdf_cut = copy.deepcopy(page_pdf)

In [9]:
# Remove all newline characters from the page content to reduce token count
for i in pages_pdf_cut:
    i.page_content = ' '.join(i.page_content.split())



In [11]:
page_pdf[0].page_content, pages_pdf_cut[0].page_content
# 375 tokens vs. 300 tokens (check at platform.openai.com/tokenizer)

('Analysis vs Analytics \nAlright! So… \nLet’s discuss the not-so-obvious differences \nbetween the terms analysis and analytics. \nDue to the similarity of the words, some people \nbelieve they share the same meaning, and thus \nuse them interchangeably. Technically, this \nisn’t correct. There is, in fact, a distinct \ndifference between the two. And the reason \nfor one often being used instead of the other \nis the lack of a transparent understanding \nof both. \nSo, let’s clear this up, shall we? \nFirst, we will start with analysis. \nConsider the following… \nYou have a huge dataset containing data of \nvarious types. Instead of tackling the entire \ndataset and running the risk of becoming overwhelmed, \nyou separate it into easier to digest chunks \nand study them individually and examine how \nthey relate to other parts. And that’s analysis \nin a nutshell. \nOne important thing to remember, however, \nis that you perform analyses on things that \nhave already happened in the

### Indexing : Document loading with DOX2TEXT Loader

In [2]:
loader_docx = Docx2txtLoader("Documents_for_indexing/introduction_to_Data_and_Data_Science.docx")

In [3]:
page_docx = loader_docx.load()

In [4]:
page_docx

[Document(metadata={'source': 'Documents_for_indexing/introduction_to_Data_and_Data_Science.docx'}, page_content="Analysis vs Analytics\n\nAlright! So…\nLet’s discuss the not-so-obvious differences\nbetween the terms analysis and analytics.\nDue to the similarity of the words, some people\nbelieve they share the same meaning, and thus\nuse them interchangeably. Technically, this\nisn’t correct. There is, in fact, a distinct\ndifference between the two. And the reason\nfor one often being used instead of the other\nis the lack of a transparent understanding\nof both.\nSo, let’s clear this up, shall we?\nFirst, we will start with analysis.\nConsider the following…\nYou have a huge dataset containing data of\nvarious types. Instead of tackling the entire\ndataset and running the risk of becoming overwhelmed,\nyou separate it into easier to digest chunks\nand study them individually and examine how\nthey relate to other parts. And that’s analysis\nin a nutshell.\nOne important thing to rem

In [5]:
for i in range(len(page_docx)):
    page_docx[i].page_content = ' '.join(page_docx[i].page_content.split())

In [6]:
len(page_docx[0].page_content)

8259

In [7]:
character_splitter = CharacterTextSplitter(
    separator=' ', 
    chunk_size=500, 
    chunk_overlap=50)

In [8]:
pages_character_split = character_splitter.split_documents(page_docx)

In [9]:
pages_character_split

[Document(metadata={'source': 'Documents_for_indexing/introduction_to_Data_and_Data_Science.docx'}, page_content='Analysis vs Analytics Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will'),
 Document(metadata={'source': 'Documents_for_indexing/introduction_to_Data_and_Data_Science.docx'}, page_content='So, let’s clear this up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to dige

In [10]:
len(pages_character_split)

19

### Indexing : Document splitting with Markdown Header Test Splitter

In [11]:
loader_docx_2 = Docx2txtLoader("Documents_for_indexing/introduction_to_Data_and_Data_Science_2.docx")
page_docx_2 = loader_docx_2.load()

In [12]:
from httpx import head


md_header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Course Title"),
        ("##", "Lecture Title")
    ]
)

In [13]:
pages_md_split = md_header_splitter.split_text(page_docx_2[0].page_content)

In [14]:
pages_md_split

[Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content="Alright! So…\nLet’s discuss the not-so-obvious differences\nbetween the terms analysis and analytics.\nDue to the similarity of the words, some people\nbelieve they share the same meaning, and thus\nuse them interchangeably. Technically, this\nisn’t correct. There is, in fact, a distinct\ndifference between the two. And the reason\nfor one often being used instead of the other\nis the lack of a transparent understanding\nof both.\nSo, let’s clear this up, shall we?\nFirst, we will start with analysis.\nConsider the following…\nYou have a huge dataset containing data of\nvarious types. Instead of tackling the entire\ndataset and running the risk of becoming overwhelmed,\nyou separate it into easier to digest chunks\nand study them individually and examine how\nthey relate to other parts. And that’s analysis\nin a nutshell.\nOne important thing to remember, however

In [15]:
for i in range(len(pages_md_split)):
    pages_md_split[i].page_content = ' '.join(pages_md_split[i].page_content.split())


In [16]:
pages_md_split

[Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content="Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to digest chunks and study them individually and examine how they relate to other parts. And that’s analysis in a nutshell. One important thing to remember, however, is that you perform

### Indexing : Text Embedding with OpenAI

In [17]:
pages_md_and_char_split = character_splitter.split_documents(pages_md_split)

In [18]:
pages_md_and_char_split

[Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis.'),
 Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to dige

In [19]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [20]:
pages_md_and_char_split[3], pages_md_and_char_split[5], pages_md_and_char_split[18]

(Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='ones. Analytics is essentially the application of logical and computational reasoning to the component parts obtained in an analysis. And in doing this you are looking for patterns and exploring what you could do with them in the future. Here, analytics branches off into two areas: qualitative analytics – this is using your intuition and experience in conjunction with the analysis to plan your next business move. And quantitative analytics – this is applying formulas and algorithms to numbers'),
 Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='styles of clothing to start selling. This would be qualitative analytics. But you might not know when to introduce the new collection. In that case, relying on past sales data and user experience data, you could predict in which month it would be be

In [21]:
vector_1 = embeddings.embed_query(pages_md_and_char_split[3].page_content)
vector_2 = embeddings.embed_query(pages_md_and_char_split[5].page_content)  
vector_3 = embeddings.embed_query(pages_md_and_char_split[18].page_content)

In [22]:
np.dot(vector_1, vector_2)

np.float64(0.8605078118583023)

In [23]:
np.dot(vector_1, vector_3)

np.float64(0.7641135414930623)

In [24]:
np.dot(vector_2, vector_3)

np.float64(0.7654245657809027)

In [25]:
np.linalg.norm(vector_1) # check the magnitude (length) of the vector

np.float64(1.0000000362284815)

In [26]:
np.linalg.norm(vector_2) 

np.float64(0.9999999874541907)

### Indexing : Creating a Chroma vectorstore 

In [27]:
len(pages_md_and_char_split)

19

In [28]:
vector_store = Chroma.from_documents(
    documents=pages_md_and_char_split,
    embedding=embeddings,
    persist_directory= './intro_to_data_science_vector_store'
)

In [29]:
vector_store_from_dir = Chroma(
    persist_directory= './intro_to_data_science_vector_store',
    embedding_function=embeddings # same embedding function as used to create the vector store
)

### Indexing : Inspecting and Managing documents in a Vectorstore

In [30]:
vector_store_from_dir.get(
    ids="da0c8b57-1414-4979-b75e-f248f0b0369b",
    include=["embeddings"]
)

{'ids': ['da0c8b57-1414-4979-b75e-f248f0b0369b'],
 'embeddings': array([[ 0.00242822, -0.0171117 ,  0.02889476, ...,  0.01816242,
         -0.01065103, -0.00424665]], shape=(1, 1536)),
 'documents': None,
 'uris': None,
 'included': ['embeddings'],
 'data': None,
 'metadatas': None}

In [31]:
added_document = Document(
    metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, 
    page_content='Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis.'
)

In [32]:
vector_store_from_dir.add_documents([added_document])

['e94fedb3-117c-43bd-ba1f-ccb08c7fd6ad']

In [33]:
vector_store_from_dir.get('9f7fb500-d9b2-4653-8ffa-cd813f5c8685')

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

In [34]:
updated_document = Document(
    metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}, 
    page_content='statistical and econometric research, where techniques like regression, cluster, and factor analysis are constantly applied. As a final note, remember the following. Should you have the relevant business and theoretical knowledge, learning a software tool is relatively easy as opposed to learning a programming language. More importantly, it will be sufficient for your need to create quick and accurate analyses. However, if your theoretical preparation is strong enough, you will find yourself')


In [35]:
vector_store_from_dir.update_document(document_id="9f7fb500-d9b2-4653-8ffa-cd813f5c8685", document=updated_document)

    

In [36]:
vector_store_from_dir.get('9f7fb500-d9b2-4653-8ffa-cd813f5c8685')

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

In [37]:
vector_store_from_dir.delete(
    ids="9f7fb500-d9b2-4653-8ffa-cd813f5c8685"
)



In [38]:
vector_store_from_dir.get('9f7fb500-d9b2-4653-8ffa-cd813f5c8685')


{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

### Retreval : Similarity search

In [48]:
questions = "What programming languages are used by data sciencists?"

In [43]:
retreved_docs = vector_store_from_dir.similarity_search(
    query=questions, 
    k=5)

In [45]:
for i in retreved_docs:
    print(f'page_content: {i.page_content} \n--------------------------------\nLecture Title: {i.metadata["Lecture Title"]}\n')

page_content: about big data? Apart from R and Python, people working in this area are often proficient in other languages like Java or Scala. These two have not been developed specifically for doing statistical analyses, however they turn out to be very useful when combining data from multiple sources. All right! Let’s finish off with machine learning. When it comes to machine learning, we often deal with big data. Thus, we need a lot of computational power, and we can expect people to use the languages 
--------------------------------
Lecture Title: Programming Languages & Software Employed in Data Science - All the Tools You Need

page_content: about big data? Apart from R and Python, people working in this area are often proficient in other languages like Java or Scala. These two have not been developed specifically for doing statistical analyses, however they turn out to be very useful when combining data from multiple sources. All right! Let’s finish off with machine learning. W

### Retreval : Maximal Marginal Relevancy Search 
Using MMR to retreve more diverse results by affecting diversity parametres and filtering

In [47]:
question_2 = "What software do data sciencists use?"

In [50]:
retreved_docs_2 = vector_store_from_dir.similarity_search(
    query=question_2, 
    k=3)

In [51]:
for i in retreved_docs_2:
    print(f'page_content: {i.page_content} \n--------------------------------\nLecture Title: {i.metadata["Lecture Title"]}\n')

page_content: is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on. Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching! 
--------------------------------
Lecture Title: Programming Languages & Software Employed in Data Science - All the Tools You Need

page_content: is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on. Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching! 
--------------------------------
Lecture Title: 

In [52]:
retreved_docs_3 = vector_store_from_dir.max_marginal_relevance_search(
    query=question_2, 
    k=3,
    lambda_mult=0.1)

In [55]:
for i in retreved_docs_3:
    print(f'page_content: {i.page_content} \n--------------------------------\nLecture Title: {i.metadata["Lecture Title"]}\n\n')

page_content: is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on. Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching! 
--------------------------------
Lecture Title: Programming Languages & Software Employed in Data Science - All the Tools You Need


page_content: the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers which is basically the way to handle big data nowadays. Power BI, SaS, Qlik, and especially Tableau are top-notch examples of software designed for business intelligence visualizations. In terms of predictive analytics, EViews is mostly used for working with econometric time-series models, and Stata—for

In [58]:
retreved_docs_4 = vector_store_from_dir.max_marginal_relevance_search(
    query=question_2, 
    k=3,
    lambda_mult=0.1,
    filter={'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'})

In [59]:
for i in retreved_docs_4:
    print(f'page_content: {i.page_content} \n--------------------------------\nLecture Title: {i.metadata["Lecture Title"]}\n\n')

page_content: is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on. Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching! 
--------------------------------
Lecture Title: Programming Languages & Software Employed in Data Science - All the Tools You Need


page_content: the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers which is basically the way to handle big data nowadays. Power BI, SaS, Qlik, and especially Tableau are top-notch examples of software designed for business intelligence visualizations. In terms of predictive analytics, EViews is mostly used for working with econometric time-series models, and Stata—for

In [60]:
retreved_docs_5 = vector_store_from_dir.max_marginal_relevance_search(
    query=question_2, 
    k=3,
    lambda_mult=0.7,
    filter={'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'})

In [61]:
for i in retreved_docs_5:
    print(f'page_content: {i.page_content} \n--------------------------------\nLecture Title: {i.metadata["Lecture Title"]}\n\n')

page_content: is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on. Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching! 
--------------------------------
Lecture Title: Programming Languages & Software Employed in Data Science - All the Tools You Need


page_content: the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers which is basically the way to handle big data nowadays. Power BI, SaS, Qlik, and especially Tableau are top-notch examples of software designed for business intelligence visualizations. In terms of predictive analytics, EViews is mostly used for working with econometric time-series models, and Stata—for

### Retreval : Vectorstore backed retrever


In [62]:
len(vector_store_from_dir.get()['documents'])

39

In [64]:
retrever = vector_store_from_dir.as_retriever(
    search_type="mmr", 
    search_kwargs={'k': 3, 'lambda_mult': 0.7})

In [65]:
retrever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x115d219f0>, search_type='mmr', search_kwargs={'k': 3, 'lambda_mult': 0.7})

In [66]:
retrevied_docs_6 = retrever.invoke(question_2)
    

In [67]:
for i in retrevied_docs_6:
    print(f'page_content: {i.page_content} \n--------------------------------\nLecture Title: {i.metadata["Lecture Title"]}\n\n')

page_content: is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on. Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching! 
--------------------------------
Lecture Title: Programming Languages & Software Employed in Data Science - All the Tools You Need


page_content: the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers which is basically the way to handle big data nowadays. Power BI, SaS, Qlik, and especially Tableau are top-notch examples of software designed for business intelligence visualizations. In terms of predictive analytics, EViews is mostly used for working with econometric time-series models, and Stata—for

### Generation : Stuffing Documents

In [69]:
new_vector_store = Chroma(
    persist_directory= './intro_to_data_science_vector_store',
    embedding_function=embeddings
)


In [70]:
len(new_vector_store.get()['documents'])

39

#### Removing all but one duplicate document in the store

In [73]:
all_ids = new_vector_store.get()['ids']

In [75]:
ids_to_delete = all_ids[21:]

In [76]:
new_vector_store.delete(ids=ids_to_delete)

In [78]:
len(new_vector_store.get()['documents'])

21

In [79]:
TEMPLATE = """ 
Answer the following questions: 
{question}

To answer this question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is from. User
the following format:
- Resource: [Lecture Title]
Where [Lecture Title] should be substituted with the title of the resource lecture.
""" 

In [100]:
prompt = PromptTemplate.from_template(TEMPLATE)

In [101]:
chat = ChatOpenAI(model="gpt-4", temperature=0, seed= 365)

In [102]:
question_3 = "What software do data sciencists use?"

In [105]:
chain = ({
    'context': retrever,
    'question' : RunnablePassthrough()
    } 
    | prompt 
    | chat 
    | StrOutputParser()
)


In [106]:
chain.invoke(question_3)

'Data scientists use a variety of software and programming languages. They often use R and Python for creating specific tools for each project. For handling the complexity of big data and its computational intensity, Hadoop is commonly used as it distributes computational tasks on multiple computers. For business intelligence visualizations, software like Power BI, SaS, Qlik, and Tableau are used. EViews is used for working with econometric time-series models, and Stata is used for academic statistical and econometric research. In addition to R and Python, data scientists working with big data may also use languages like Java or Scala. \n\n- Resource: Programming Languages & Software Employed in Data Science - All the Tools You Need'